In [ ]:
#|default_exp models.MLP

# MLP

This is an unofficial PyTorch implementation created by Ignacio Oguiza (oguiza@timeseriesAI.co) based on:

Fawaz, H. I., Forestier, G., Weber, J., Idoumghar, L., & Muller, P. A.
(2019). <span style="color:dodgerblue">**Deep learning for time series classification: a review**</span>. Data Mining and
Knowledge Discovery, 33(4), 917-963. 

Official MLP TensorFlow implementation:
https://github.com/hfawaz/dl-4-tsc/blob/master/classifiers/mlp.py

In [ ]:
#|export
from tsai.imports import *
from fastai.layers import *
from tsai.models.layers import *

In [ ]:
#|export
class MLP(Module):
    def __init__(self, c_in, c_out, seq_len, layers=[500,500,500], ps=[0.1, 0.2, 0.2], act=nn.ReLU(inplace=True),
                 use_bn=False, bn_final=False, lin_first=False, fc_dropout=0., y_range=None):
        layers, ps = L(layers), L(ps)
        if len(ps) <= 1: ps = ps * len(layers)
        assert len(layers) == len(ps), '#layers and #ps must match'
        self.flatten = Reshape()
        nf = [c_in * seq_len] + layers
        self.mlp = nn.ModuleList()
        for i in range(len(layers)): self.mlp.append(LinBnDrop(nf[i], nf[i+1], bn=use_bn, p=ps[i], act=get_act_fn(act), lin_first=lin_first))
        _head = [LinBnDrop(nf[-1], c_out, bn=bn_final, p=fc_dropout)]
        if y_range is not None: _head.append(SigmoidRange(*y_range))
        self.head = nn.Sequential(*_head)

    def forward(self, x):
        x = self.flatten(x)
        for mlp in self.mlp: x = mlp(x)
        return self.head(x)

In [ ]:
bs = 16
nvars = 3
seq_len = 128
c_out = 2
xb = torch.rand(bs, nvars, seq_len)
model = MLP(nvars, c_out, seq_len)
test_eq(model(xb).shape, (bs, c_out))
model

MLP(
  (flatten): Reshape(bs)
  (mlp): ModuleList(
    (0): LinBnDrop(
      (0): Dropout(p=0.1, inplace=False)
      (1): Linear(in_features=384, out_features=500, bias=True)
      (2): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=500, out_features=500, bias=True)
      (2): ReLU(inplace=True)
    )
    (2): LinBnDrop(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=500, out_features=500, bias=True)
      (2): ReLU(inplace=True)
    )
  )
  (head): Sequential(
    (0): LinBnDrop(
      (0): Linear(in_features=500, out_features=2, bias=True)
    )
  )
)

In [ ]:
#|eval: false
#|hide
from tsai.export import get_nb_name; nb_name = get_nb_name(locals())
from tsai.imports import create_scripts; create_scripts(nb_name)

<IPython.core.display.Javascript object>

/Users/nacho/notebooks/tsai/nbs/037_models.MLP.ipynb saved at 2023-03-15 11:58:07
Correct notebook to script conversion! 😃
Wednesday 15/03/23 11:58:09 CET
